In [ ]:
%gui qt

In [1]:
import napari
import sys
Linux = True
if Linux:
    sys.path.insert(0,'/home/georgeos/Documents/GitHub/SYMPTOMM2')
else:
    sys.path.insert(0,'C:\\Users\\gh464\\Documents\\GitHub\\SYMPTOMM2\\')
from SYMPTOMM.cell import Cell
from SYMPTOMM.scene_functions import create_space, step_and_update
from SYMPTOMM.trench_geometry import trench_creator
from SYMPTOMM.phase_contrast_drawing import *
import tifffile
import numpy as np
from skimage.exposure import rescale_intensity
from skimage.transform import rescale, resize, downscale_local_mean
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from skimage.exposure import rescale_intensity
from ipywidgets import interactive
import os
from PIL import Image
import skimage
from tqdm.notebook import tqdm
import gc
from SYMPTOMM.PSF import get_phase_contrast_kernel, get_condensers
real_image = tifffile.imread("C:\\Users\\gh464\\Documents\\top_trenches_PC\\trench_{}\T_{}.tif".format(
    str(np.random.randint(1,56)).zfill(2),
    str(np.random.randint(10,20)).zfill(3)))

#viewer = napari.view_image(real_image)


FileNotFoundError: [Errno 2] No such file or directory: '/home/georgeos/Documents/GitHub/SYMPTOMM2/prototyping/C:\\Users\\gh464\\Documents\\top_trenches_PC\\trench_26\\T_019.tif'

In [2]:
draw_scene

<function SYMPTOMM.phase_contrast_drawing.draw_scene(cell_properties, do_transformation, mask_threshold, space_size, offset, label_masks)>

In [2]:
#real_image = tifffile.imread("/home/georgeos/Storage/Dropbox (Cambridge University)/Phase/Lane_02_pos_002_trench_x_1963_y_0067_w_0046_h_0460_c_Phase.tif")[0][:300,:]


### Scope props

In [3]:
condensers = get_condensers()
W, R, diameter = condensers["Ph2"]
radius=50
#F = 5
λ = 0.75
resize_amount = 3
pix_mic_conv = 0.10837 ##0.108379937 micron/pix for 60x, 0.0655 for 100x
scale = pix_mic_conv / resize_amount 
min_sigma = 0.42*0.6/6 / scale # micron#
sigma=min_sigma
NA=0.97
n = 1
kernel_params = (R,W,radius,scale,NA,n,sigma,λ)

#kernel_params = (R,W,radius,scale,F,sigma,λ)


In [4]:
kernel_params = (R,W,radius,scale,NA,n,sigma,λ)


### Do large cells

In [31]:
sim_length = 400
cell_timeseries, space = run_simulation(
    trench_length=17.5, 
    trench_width=1.5, 
    cell_max_length=4.7, 
    cell_width=1.15, 
    sim_length = sim_length, 
    pix_mic_conv = pix_mic_conv,
    gravity=-10,
    phys_iters=25
) # growth phase
main_segments = get_trench_segments(space)
ID_props = generate_curve_props(cell_timeseries)
cell_timeseries_properties = Parallel(n_jobs=-1)(
    delayed(gen_cell_props_for_draw)(a, ID_props) for a in tqdm(cell_timeseries, desc='Timeseries Properties'))
do_transformation = True
offset = 30
mask_threshold = 18
label_masks = True
space_size = get_space_size(cell_timeseries_properties)
scenes = Parallel(n_jobs=-1)(delayed(draw_scene)(
    cell_properties, do_transformation, mask_threshold, space_size, offset, label_masks) for cell_properties in tqdm(cell_timeseries_properties, desc='Scene Draw:'))








Simulation Progress:   0%|                                                                     | 0/650 [00:00<?, ?it/s]






Simulation Progress:   7%|███▉                                                       | 43/650 [00:00<00:01, 401.87it/s]






Simulation Progress:  13%|███████▌                                                   | 84/650 [00:00<00:02, 277.84it/s]






Simulation Progress:  18%|██████████▏                                               | 114/650 [00:00<00:02, 200.36it/s]






Simulation Progress:  21%|████████████▏                                             | 137/650 [00:00<00:03, 163.33it/s]






Simulation Progress:  24%|█████████████▉                                            | 156/650 [00:00<00:03, 139.74it/s]






Simulation Progress:  26%|███████████████▎                                          | 172/650 [00:01<00:03, 132.26it/s]






Simulation Progress:  29%|████████████████▌                                         | 186/650 [00:01<00:03, 122

Timeseries Properties:   0%|          | 0/399 [00:00<?, ?it/s]

Scene Draw::   0%|          | 0/399 [00:00<?, ?it/s]

In [19]:
media_multiplier=30
cell_multiplier=1
device_multiplier=-50
y_border_expansion_coefficient = 2
x_border_expansion_coefficient = 3




temp_expanded_scene, temp_expanded_scene_no_cells, temp_expanded_mask = generate_PC_OPL(
   main_segments=main_segments,
    offset=offset,
    scene = scenes[0][0],
    mask = scenes[0][1],
    media_multiplier=media_multiplier,
    cell_multiplier=cell_multiplier,
    device_multiplier=cell_multiplier,
    y_border_expansion_coefficient = y_border_expansion_coefficient,
    x_border_expansion_coefficient = x_border_expansion_coefficient
)



### Generate temporary image to make same shape
temp_kernel = get_phase_contrast_kernel(*kernel_params)
convolved = convolve_rescale(temp_expanded_scene, temp_kernel, 1/resize_amount, rescale_int = True)
real_resize, expanded_resized = make_images_same_shape(real_image,convolved, rescale_int=True)



C:\Users\gh464\Documents\GitHub\SYMPTOMM2\SYMPTOMM\PSF.py:104: RuntimeWarning: invalid value encountered in true_divide
  kernel1 = 2*jv(1,rr)/(rr)
C:\Users\gh464\Documents\GitHub\SYMPTOMM2\SYMPTOMM\PSF.py:107: RuntimeWarning: invalid value encountered in true_divide
  kernel2 = 2*(R-W)**2/R**2 * jv(1,(R-W)**2/R**2 * rr)/rr


In [20]:
viewer = napari.view_image(real_resize)
media_label = viewer.add_labels(np.zeros(real_resize.shape).astype(int), name = "media")
cell_label = viewer.add_labels(np.zeros(real_resize.shape).astype(int), name = "cell")
device_label = viewer.add_labels(np.zeros(real_resize.shape).astype(int), name = "device")

C:\Users\gh464\anaconda3\envs\SYMPTOMM\lib\site-packages\napari\_qt\qt_main_window.py:420: RuntimeWarning: The window geometry settings could not be loaded due to the following error: cannot unpack non-iterable NoneType object
  self.show()


In [21]:
real_media_mean = real_resize[np.where(media_label.data)].mean();
real_cell_mean = real_resize[np.where(cell_label.data)].mean(); 
real_device_mean = real_resize[np.where(device_label.data)].mean()
real_means = np.array((real_media_mean, real_cell_mean, real_device_mean))

real_media_var = real_resize[np.where(media_label.data)].var();
real_cell_var = real_resize[np.where(cell_label.data)].var(); 
real_device_var = real_resize[np.where(device_label.data)].var()
real_vars = np.array((real_media_var, real_cell_var, real_device_var))

image_params = (real_media_mean, real_cell_mean, real_device_mean, real_means, real_media_var, real_cell_var, real_device_var, real_vars)

<ipython-input-21-c6a8b6c5825e>:1: RuntimeWarning: Mean of empty slice.
  real_media_mean = real_resize[np.where(media_label.data)].mean();
C:\Users\gh464\anaconda3\envs\SYMPTOMM\lib\site-packages\numpy\core\_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
<ipython-input-21-c6a8b6c5825e>:2: RuntimeWarning: Mean of empty slice.
  real_cell_mean = real_resize[np.where(cell_label.data)].mean();
<ipython-input-21-c6a8b6c5825e>:3: RuntimeWarning: Mean of empty slice.
  real_device_mean = real_resize[np.where(device_label.data)].mean()
<ipython-input-21-c6a8b6c5825e>:6: RuntimeWarning: Degrees of freedom <= 0 for slice
  real_media_var = real_resize[np.where(media_label.data)].var();
C:\Users\gh464\anaconda3\envs\SYMPTOMM\lib\site-packages\numpy\core\_methods.py:194: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(
C:\Users\gh464\anaconda3\envs\SYMPTOMM\lib\site-packages\numpy\core\_methods.p

In [33]:
mean_error = []
media_error = []
cell_error = []
device_error = []

mean_var_error = []
media_var_error = []
cell_var_error = []
device_var_error = []

error_params = (mean_error,media_error,cell_error,device_error,mean_var_error,media_var_error,cell_var_error,device_var_error)

params = interactive(
    generate_test_comparison,
    media_multiplier=(-300,300,1),
    cell_multiplier=(-30,30,0.01),
    device_multiplier=(-300,300,1),
    sigma=(min_sigma,min_sigma*20, min_sigma/20),
    scene_no = (0,sim_length-2,1),
    noise_var=(0,0.01, 0.0001),
    scale=(scale,scale,scale),
    match_histogram = [True, False],
    match_noise = [True, False],
    offset=fixed(offset),
    main_segments = fixed(main_segments),
    debug_plot=fixed(True),
    scenes = fixed(scenes),
    kernel_params = fixed(kernel_params),
    resize_amount = fixed(resize_amount), 
    real_image = fixed(real_image),
    image_params = fixed(image_params),
    error_params = fixed(error_params),
    x_border_expansion_coefficient = fixed(x_border_expansion_coefficient),
    y_border_expansion_coefficient = fixed(y_border_expansion_coefficient)
);

In [34]:
params

interactive(children=(IntSlider(value=0, description='media_multiplier', max=300, min=-300), FloatSlider(value…

In [36]:
generate_training_data(interactive_output = params, sample_amount = 0.12, randomise_hist_match = True, randomise_noise_match = False, sim_length = sim_length, burn_in = 0, n_samples =  5000, save_dir = "C:\\Users\\gh464\Documents\Ph2 training data\\bent_noisematched_allsize_emptytrench\\")

Sample generation:   0%|          | 0/5000 [00:00<?, ?it/s]

KeyboardInterrupt: 

### Do medium cells

In [37]:
sim_length = 400
cell_timeseries, space = run_simulation(
    trench_length=17.5, 
    trench_width=1.5, 
    cell_max_length=4.8, 
    cell_width=1.15, 
    sim_length = sim_length, 
    pix_mic_conv = pix_mic_conv,
    gravity=-7,
    phys_iters=25
) # growth phase
main_segments = get_trench_segments(space)
ID_props = generate_curve_props(cell_timeseries)
cell_timeseries_properties = Parallel(n_jobs=-1)(
    delayed(gen_cell_props_for_draw)(a, ID_props) for a in tqdm(cell_timeseries, desc='Timeseries Properties'))
do_transformation = True
offset = 30
mask_threshold = 18
label_masks = True
space_size = get_space_size(cell_timeseries_properties)
scenes = Parallel(n_jobs=-1)(delayed(draw_scene)(
    cell_properties, do_transformation, mask_threshold, space_size, offset, label_masks) for cell_properties in tqdm(cell_timeseries_properties, desc='Scene Draw:'))


Simulation Progress: 100%|███████████████████████████████████████████████████████████| 650/650 [00:13<00:00, 46.72it/s]


Timeseries Properties:   0%|          | 0/399 [00:00<?, ?it/s]

Scene Draw::   0%|          | 0/399 [00:00<?, ?it/s]

In [43]:
mean_error = []
media_error = []
cell_error = []
device_error = []

mean_var_error = []
media_var_error = []
cell_var_error = []
device_var_error = []

error_params = (mean_error,media_error,cell_error,device_error,mean_var_error,media_var_error,cell_var_error,device_var_error)

params = interactive(
    generate_test_comparison,
    media_multiplier=(-300,300,1),
    cell_multiplier=(-30,30,0.01),
    device_multiplier=(-300,300,1),
    sigma=(min_sigma,min_sigma*20, min_sigma/20),
    scene_no = (0,sim_length-2,1),
    noise_var=(0,0.01, 0.0001),
    scale=(scale,scale,scale),
    match_histogram = [True, False],
    match_noise = [True, False],
    offset=fixed(offset),
    main_segments = fixed(main_segments),
    debug_plot=fixed(True),
    scenes = fixed(scenes),
    kernel_params = fixed(kernel_params),
    resize_amount = fixed(resize_amount), 
    real_image = fixed(real_image),
    image_params = fixed(image_params),
    error_params = fixed(error_params),
    x_border_expansion_coefficient = fixed(x_border_expansion_coefficient),
    y_border_expansion_coefficient = fixed(y_border_expansion_coefficient)
);

In [44]:
params

interactive(children=(IntSlider(value=0, description='media_multiplier', max=300, min=-300), FloatSlider(value…

In [46]:
generate_training_data(interactive_output = params, sample_amount = 0.075, randomise_hist_match = False, randomise_noise_match = False, sim_length = sim_length, burn_in = 0, n_samples =  5000, save_dir = "C:\\Users\\gh464\Documents\Ph2 training data\\bent_noisematched_allsize_emptytrench\\")

Sample generation:   0%|          | 0/5000 [00:00<?, ?it/s]

KeyboardInterrupt: 

## Do small cells

In [47]:
sim_length = 600
cell_timeseries, space = run_simulation(
    trench_length=17.5, 
    trench_width=1.5, 
    cell_max_length=2.8, 
    cell_width=1.05, 
    sim_length = sim_length, 
    pix_mic_conv = pix_mic_conv,
    gravity=-7,
    phys_iters=25
) # growth phase
main_segments = get_trench_segments(space)
ID_props = generate_curve_props(cell_timeseries)
cell_timeseries_properties = Parallel(n_jobs=-1)(
    delayed(gen_cell_props_for_draw)(a, ID_props) for a in tqdm(cell_timeseries, desc='Timeseries Properties'))
do_transformation = True
offset = 30
mask_threshold = 18
label_masks = True
space_size = get_space_size(cell_timeseries_properties)
scenes = Parallel(n_jobs=-1)(delayed(draw_scene)(
    cell_properties, do_transformation, mask_threshold, space_size, offset, label_masks) for cell_properties in tqdm(cell_timeseries_properties, desc='Scene Draw:'))



Simulation Progress:   0%|                                                                     | 0/850 [00:00<?, ?it/s]

Simulation Progress:   3%|█▌                                                         | 23/850 [00:00<00:03, 221.16it/s]

Simulation Progress:   5%|███▏                                                       | 46/850 [00:00<00:03, 206.01it/s]

Simulation Progress:   8%|████▋                                                      | 67/850 [00:00<00:04, 158.15it/s]

Simulation Progress:  10%|█████▊                                                     | 84/850 [00:00<00:05, 142.02it/s]

Simulation Progress:  12%|██████▊                                                    | 99/850 [00:00<00:06, 115.35it/s]

Simulation Progress:  13%|███████▊                                                   | 112/850 [00:00<00:07, 95.30it/s]

Simulation Progress:  14%|████████▌                                                  | 123/850 [00:01<00:08, 88.26it/s]

Simulation Progress:  16%|████

Timeseries Properties:   0%|          | 0/599 [00:00<?, ?it/s]

Scene Draw::   0%|          | 0/599 [00:00<?, ?it/s]

In [48]:
mean_error = []
media_error = []
cell_error = []
device_error = []

mean_var_error = []
media_var_error = []
cell_var_error = []
device_var_error = []

error_params = (mean_error,media_error,cell_error,device_error,mean_var_error,media_var_error,cell_var_error,device_var_error)

params = interactive(
    generate_test_comparison,
    media_multiplier=(-300,300,1),
    cell_multiplier=(-30,30,0.01),
    device_multiplier=(-300,300,1),
    sigma=(min_sigma,min_sigma*20, min_sigma/20),
    scene_no = (0,sim_length-2,1),
    noise_var=(0,0.01, 0.0001),
    scale=(scale,scale,scale),
    match_histogram = [True, False],
    match_noise = [True, False],
    offset=fixed(offset),
    main_segments = fixed(main_segments),
    debug_plot=fixed(True),
    scenes = fixed(scenes),
    kernel_params = fixed(kernel_params),
    resize_amount = fixed(resize_amount), 
    real_image = fixed(real_image),
    image_params = fixed(image_params),
    error_params = fixed(error_params),
    x_border_expansion_coefficient = fixed(x_border_expansion_coefficient),
    y_border_expansion_coefficient = fixed(y_border_expansion_coefficient)
);

In [49]:
params

interactive(children=(IntSlider(value=0, description='media_multiplier', max=300, min=-300), FloatSlider(value…

In [50]:
generate_training_data(interactive_output = params, sample_amount = 0.075, randomise_hist_match = False, randomise_noise_match = False, sim_length = sim_length, burn_in = 0, n_samples =  500, save_dir = "C:\\Users\\gh464\Documents\Ph2 training data\\bent_noisematched_allsize_emptytrench\\")

Sample generation:   0%|          | 0/500 [00:00<?, ?it/s]

### Do a few very small cells

In [24]:
sim_length = 100
cell_timeseries, space = run_simulation(
    trench_length=17.5, 
    trench_width=1.5, 
    cell_max_length=1.7, 
    cell_width=1.05, 
    sim_length = sim_length, 
    pix_mic_conv = pix_mic_conv,
    gravity=-1,
    phys_iters=39
) # growth phase
main_segments = get_trench_segments(space)
ID_props = generate_curve_props(cell_timeseries)
cell_timeseries_properties = Parallel(n_jobs=-1)(
    delayed(gen_cell_props_for_draw)(a, ID_props) for a in tqdm(cell_timeseries, desc='Timeseries Properties'))
do_transformation = True
offset = 40
mask_threshold = 18
label_masks = True
space_size = get_space_size(cell_timeseries_properties)


Simulation Progress: 100%|███████████████████████████████████████████████████████████| 350/350 [00:11<00:00, 29.33it/s]


Timeseries Properties:   0%|          | 0/99 [00:00<?, ?it/s]

In [25]:
scenes = Parallel(n_jobs=4)(delayed(draw_scene)(
    cell_properties, do_transformation, mask_threshold, space_size, offset, label_masks) for cell_properties in tqdm(cell_timeseries_properties, desc='Scene Draw:'))

Scene Draw::   0%|          | 0/99 [00:00<?, ?it/s]

In [17]:
gc.collect()

1659

In [27]:
mean_error = []
media_error = []
cell_error = []
device_error = []

mean_var_error = []
media_var_error = []
cell_var_error = []
device_var_error = []

error_params = (mean_error,media_error,cell_error,device_error,mean_var_error,media_var_error,cell_var_error,device_var_error)

params = interactive(
    generate_test_comparison,
    media_multiplier=(-300,300,1),
    cell_multiplier=(-30,30,0.01),
    device_multiplier=(-300,300,1),
    sigma=(min_sigma,min_sigma*20, min_sigma/20),
    scene_no = (0,sim_length-2,1),
    noise_var=(0,0.01, 0.0001),
    scale=(scale,scale,scale),
    match_histogram = [True, False],
    match_noise = [True, False],
    offset=fixed(offset),
    main_segments = fixed(main_segments),
    debug_plot=fixed(True),
    scenes = fixed(scenes),
    kernel_params = fixed(kernel_params),
    resize_amount = fixed(resize_amount), 
    real_image = fixed(real_image),
    image_params = fixed(image_params),
    error_params = fixed(error_params),
    x_border_expansion_coefficient = fixed(x_border_expansion_coefficient),
    y_border_expansion_coefficient = fixed(y_border_expansion_coefficient)
);

In [28]:
params

interactive(children=(IntSlider(value=0, description='media_multiplier', max=300, min=-300), FloatSlider(value…

In [35]:
import os
def generate_training_data(interactive_output, sample_amount, randomise_hist_match, randomise_noise_match, sim_length, burn_in, n_samples, save_dir):
    #media_multiplier, cell_multiplier, device_multiplier, sigma, scene_no, scale, match_histogram, match_noise, offset, debug_plot, noise_var = list(interactive_output.kwargs.values())
    media_multiplier, cell_multiplier, device_multiplier, sigma, scene_no, scale, match_histogram, match_noise, offset, debug_plot, noise_var, main_segments, scenes, kernel_params, resize_amount, real_image, image_params, error_params, x_border_expansion_coefficient,y_border_expansion_coefficient = list(interactive_output.kwargs.values())
    debug_plot = False
    try:
        os.mkdir(save_dir)
    except:
        pass
    try:
        os.mkdir(save_dir + "/convolutions")
    except:
        pass
    try:
        os.mkdir(save_dir + "/masks")
    except:
        pass
        
    current_file_num = len(os.listdir(save_dir+"/convolutions"))
    #for z in range(n_samples):
    def generate_samples(z):
        _media_multiplier = np.random.uniform(1-sample_amount,1+sample_amount) * media_multiplier
        _cell_multiplier = np.random.uniform(1-sample_amount,1+sample_amount) * cell_multiplier
        _device_multiplier = np.random.uniform(1-sample_amount,1+sample_amount) * device_multiplier
        _sigma = np.random.uniform(1-sample_amount,1+sample_amount) * sigma
        _scene_no = np.random.randint(burn_in,sim_length-2)
        _noise_var = np.random.uniform(1-sample_amount,1+sample_amount) * noise_var
        if randomise_hist_match:
            _match_histogram = np.random.choice([True, False])
        else:
            _match_histogram = match_histogram
        if randomise_noise_match:
            _match_noise = np.random.choice([True, False])
        else:
            _match_noise = match_noise
        
        syn_image, mask = generate_test_comparison(_media_multiplier, _cell_multiplier, _device_multiplier, _sigma, _scene_no, scale, _match_histogram, match_noise, offset, debug_plot, noise_var, main_segments, scenes, kernel_params, resize_amount, real_image, image_params, error_params, x_border_expansion_coefficient,y_border_expansion_coefficient)
        
        syn_image = Image.fromarray(skimage.img_as_uint(rescale_intensity(syn_image)))
        syn_image.save("{}/convolutions/synth_{}.tif".format(save_dir, str(z).zfill(5)))
        mask = Image.fromarray(mask.astype(np.uint8))
        mask.save("{}/masks/synth_{}.tif".format(save_dir, str(z).zfill(5)))        
    ## TODO: change parallel if not using GPU
    Parallel(n_jobs=1)(delayed(generate_samples)(z) for z in tqdm(range(current_file_num,n_samples+current_file_num), desc="Sample generation"))

In [36]:
generate_training_data(interactive_output = params, sample_amount = 0.075, randomise_hist_match = False, randomise_noise_match = False, sim_length = sim_length, burn_in = 0, n_samples =  200, save_dir = "C:\\Users\\gh464\Documents\Ph2 training data\\bent_noisematched_allsize_emptytrench\\")

Sample generation:   0%|          | 0/200 [00:00<?, ?it/s]

### Do a few empty trenches with different sizes

Rememebr to set `cell_multiplier` to 0

In [38]:
params

interactive(children=(IntSlider(value=26, description='media_multiplier', max=300, min=-300), FloatSlider(valu…

In [50]:
generate_training_data(interactive_output = params, sample_amount = 0.1, randomise_hist_match = False, randomise_noise_match = False, sim_length = sim_length, burn_in = 0, n_samples =  50, save_dir = "C:\\Users\\gh464\Documents\Ph2 training data\\bent_noisematched_allsize_emptytrench\\")

Sample generation:   0%|          | 0/50 [00:00<?, ?it/s]